In [10]:
import numpy as np 
import pandas as pd 
import os
import cv2

In [11]:
from dataclasses import dataclass
@dataclass()
class Star:
  name: str
  position: np.ndarray
  
def read_Star_data(star_file):
  star_data = pd.read_csv(star_file,header=None)
  star = []
  star_name_list = []
  for _ in star_data.iterrows():
    if _[0]%3 == 0:
      starname = _[1][1].rsplit('-',maxsplit=1)[0].replace(' ','')
      star_name_list.append(starname)
    if _[0]%3 == 1:
      star_temp = Star(name=starname, position=np.array([_[1][1], _[1][2], _[1][3]]).astype(np.float64))
      star.append(star_temp)
  return star, star_name_list

In [12]:
@dataclass()
class Sensor:
  time: pd.Timestamp
  q: np.ndarray
  position_a: np.ndarray
  position: np.ndarray

def read_sensor_data(sensor_file):
  sensor_data = pd.read_csv(sensor_file)
  sensor = []
  sensor_data['Time (UTCG)'] = pd.to_datetime(sensor_data['Time (UTCG)'])
  time_ls = sensor_data['Time (UTCG)'].tolist()
  for index, row in sensor_data.iterrows():
    sensor.append(Sensor(
      time = row[0],
      q = row[7:11].to_numpy(dtype=np.float64),
      position_a = row[1:4].to_numpy(dtype=np.float64),
      position = np.zeros(3)
    ))
  return sensor,time_ls

In [ ]:
star_uv = pd.read_csv('star_data.csv')
# sensor_attitude = pd.read_csv('data/sensor_attitude.csv')
sensor, time_ls = read_sensor_data('data/mysenior_sensor_data_output.csv')
star, star_name_list = read_Star_data('data/reduced_star.csv')
time_list = star_uv['time'].tolist()
star_uv['time'] = pd.to_datetime(star_uv['time'])


In [26]:
class StarMatch:
  def __init__(self, name, index, u, v):
    self.name = name
    self.index = index
    self.u = u
    self.v = v

In [27]:
def star_match(uv_dict, star_data, sensor_q, sensor_pos_est):
  #参数：uv_dict:字典，键为编号，值为uv值  debris_dict:字典，键为编号，值为debris名字str  debris_data:所有debris在此刻的对象，长度为debris的数目  sensor_q:传感器姿态数据  sensor_pos_est:传感器位置数据，预估，有误差
  #返回：debris_dict:字典，键为编号，值为debris名字str；error,当匹配失败时，返回一个误差
  f = 0.01413
  dh = 0.000006
  dv = 0.000006
  H = 700
  star_dict = {}
  if sensor_q[3] < 0:
    sensor_q = -sensor_q
  q0, q1, q2, q3 = sensor_q
  Msi = np.array([[q3**2+q0**2-q1**2-q2**2, 2*q3*q2+2*q0*q1,     -2*q3*q1+2*q0*q2],
                    [-2*q3*q2+2*q0*q1,    q3**2-q0**2+q1**2-q2**2, 2*q3*q0+2*q1*q2],
                    [2*q3*q1+2*q0*q2,     -2*q3*q0+2*q1*q2,    q3**2-q0**2-q1**2+q2**2]])
  match_ls = [] #存放计算的可能的目标碎片，目前只有碎片名称但没有碎片编号
  for star0 in star_data:
    name = star0.name
    vec = star0.position
    A = np.dot(Msi, vec)
    u = f*A[0]/(dh*A[2])+H
    v = f*A[1]/(dv*A[2])+H
    if u < -H or u > H*3 or v < -H or v > H*3:
      continue
    match0 = StarMatch(name, -1, v, u) # 这里uv交换一个位置，因为在图像中，x轴是v轴，y轴是u轴
    match_ls.append(match0)
  # match_ls, u_range, v_range = uv_reduce(match_ls, H)
  for index, uv_true in uv_dict.items():
    u,v = uv_true
    distance_ls = []
    for match0 in match_ls:
      distance_ls.append(np.linalg.norm(np.array([u,v])-np.array([match0.u, match0.v])))
    min_distance = min(distance_ls)
    # print(min_distance,distance_ls.index(min_distance),index)
    match_ls[distance_ls.index(min_distance)].index = index
  for match0 in match_ls:
    if match0.index == -1:
      continue
    star_dict[match0.index] = match0.name
 
  return star_dict

In [30]:
uv_dict_ls = []
for time0 in range(50,100):
  print(time0)
  uv_dict = {}
  for index, row in star_uv.iterrows():
    if row[0] == time_ls[time0]:
      index = int(row[3])
      uv_dict[index] = [row[1], row[2]]
  sensor_q = sensor[time0].q
  # sensor_pos_est = sensor[time0].position_a + np.random.normal(0, 0.1, 3) # 加噪声
  sensor_pos_est = sensor[time0].position_a
  star_data = []
  star_dict = star_match(uv_dict, star, sensor_q, sensor_pos_est)
  uv_dict_ls.append(star_dict)

50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [31]:
final_dict = {}
uv_dict = {}
for star_dict in uv_dict_ls:
  for index, name in star_dict.items():
    if index in final_dict:
      final_dict[index].append(name)
    else:
      final_dict[index] = [name]
for index, name in final_dict.items():
  unique_name = list(set(name))
  frequency=[]
  for i in unique_name:
    frequency.append(name.count(i))
    print(index, i, name.count(i))
  uv_dict[index] = unique_name[frequency.index(max(frequency))]
print(uv_dict)

36 Star-61468 50
19 Star-61789 50
17 Star-62683 34
38 Star-62867 50
29 Star-62896 50
43 Star-63724 28
41 Star-63945 50
44 Star-64004 50
2 Star-64166 50
28 Star-64408 50
12 Star-64803 50
4 Star-64962 50
33 Star-65936 50
21 Star-67153 50
22 Star-67244 50
37 Star-68245 2
37 Star-67464 48
46 Star-68002 50
40 Star-68523 29
40 Star-68282 21
7 Star-68895 50
32 Star-68933 50
39 Star-70300 3
39 Star-71352 47
24 Star-70753 50
30 Star-72571 50
42 Star-63724 22
15 Star-62683 16
13 Star-67786 14
34 Star-67464 2
{36: 'Star-61468', 19: 'Star-61789', 17: 'Star-62683', 38: 'Star-62867', 29: 'Star-62896', 43: 'Star-63724', 41: 'Star-63945', 44: 'Star-64004', 2: 'Star-64166', 28: 'Star-64408', 12: 'Star-64803', 4: 'Star-64962', 33: 'Star-65936', 21: 'Star-67153', 22: 'Star-67244', 37: 'Star-67464', 46: 'Star-68002', 40: 'Star-68523', 7: 'Star-68895', 32: 'Star-68933', 39: 'Star-71352', 24: 'Star-70753', 30: 'Star-72571', 42: 'Star-63724', 15: 'Star-62683', 13: 'Star-67786', 34: 'Star-67464'}


In [32]:
columns=['time','name','target_index','u','v','x','y','z']
visible_star = []
for index, row in star_uv.iterrows():
  if row[3] in uv_dict:
    name = uv_dict[row[3]]
    time = row[0]
    target_index = row[3]
    x,y,z = star[star_name_list.index(name)].position
    u,v = row[1],row[2]
    visible_star.append({'time':time,'name':name,'target_index':target_index,'u':u,'v':v,'x':x,'y':y,'z':z})
visible_star = pd.DataFrame(visible_star, columns=columns)
visible_star.to_csv('visible_star.csv',index=False)
print(visible_star)

                         time        name  target_index            u  \
0     2023-04-25 16:00:05.000  Star-64166             2   196.285714   
1     2023-04-25 16:00:05.000  Star-64962             4   203.714286   
2     2023-04-25 16:00:05.000  Star-68895             7   324.000000   
3     2023-04-25 16:00:05.000  Star-64803            12   499.000000   
4     2023-04-25 16:00:05.000  Star-67786            13   525.714286   
...                       ...         ...           ...          ...   
20862 2023-04-25 16:01:39.900  Star-68933            32  1072.500000   
20863 2023-04-25 16:01:39.900  Star-67464            34  1113.285714   
20864 2023-04-25 16:01:39.900  Star-65936            33  1112.875000   
20865 2023-04-25 16:01:39.900  Star-67464            37  1196.500000   
20866 2023-04-25 16:01:39.900  Star-71352            39  1331.285714   

                v         x         y         z  
0      910.000000 -0.878264 -0.272935 -0.392631  
1      547.000000 -0.865354 -0.3103